In [20]:
import efficientnet_pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import os
import glob
import time
import shutil
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

In [21]:
DATA_FOLDER = './data'
BATCH_SIZE = 4

filenames = sorted(os.listdir(DATA_FOLDER))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [22]:
# Show labels of every class
with open(os.path.join(DATA_FOLDER, 'labels.txt'), "r") as f:
    classes_text = f.read()
classes = classes_text.split('\n')
print(classes_text)

1 гривня (зразка 2006 р)
1 гривня (зразка 2006 р, зворотнiй бiк)
2 гривнi (зразка 2004 р)
2 гривнi (зразка 2004 р, зворотнiй бiк)
5 гривень (зразка 2004 р)
5 гривень (зразка 2004 р, зворотнiй бiк)
10 гривень (зразка 2004 р)
10 гривень (зразка 2004 р, зворотнiй бiк)
20 гривень (зразка 2003 р)
20 гривень (зразка 2003 р, зворотнiй бiк)
50 гривень (зразка 2004 р)
50 гривень (зразка 2004 р, зворотнiй бiк)
100 гривень (зразка 2005 р)
100 гривень (зразка 2005 р, зворотнiй бiк)
200 гривень (зразка 2007 р)
200 гривень (зразка 2007 р, зворотнiй бiк)
500 гривень (зразка 2006 р)
500 гривень (зразка 2006 р, зворотнiй бiк)
1000 гривень (зразка 2019 р)
1000 гривень (зразка 2019 р, зворотнiй бiк)
500 гривень (зразка 2015 р)
500 гривень (зразка 2015 р, зворотнiй бiк)
100 гривень (зразка 2014 р)
100 гривень (зразка 2014 р, зворотнiй бiк)
20 гривень (зразка 2018 р)
20 гривень (зразка 2018 р, зворотнiй бiк)
50 гривень (зразка 2019 р)
50 гривень (зразка 2019 р, зворотнiй бiк)
200 гривень (зразка 2019 р)
20

In [23]:
# Gathering paths of every image in dataset.
data = []

filenames = [fname for fname in filenames if fname != 'labels.txt']
for fname in filenames:
    images = glob.glob(os.path.join(DATA_FOLDER, fname, '*.png'))
    images += glob.glob(os.path.join(DATA_FOLDER, fname, '*.jpg'))
    images += glob.glob(os.path.join(DATA_FOLDER, fname, '*.jpeg'))
    data.append((fname, images))

print(data[0][0], data[0][1][0])

01-1a ./data/01-1a/tf-1565952427481.png


In [24]:
# Splitting all the images into train, dev, test sets.

train_dir = './train'
dev_dir = './dev'
test_dir = './test'
random.seed(42)

if not os.path.isdir(train_dir) or not os.path.isdir(dev_dir) or not os.path.isdir(test_dir):

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(dev_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    for name, images in data:
        os.makedirs(os.path.join(train_dir, name))
        os.makedirs(os.path.join(dev_dir, name))
        os.makedirs(os.path.join(test_dir, name))
        
        train_end = int(0.8 * len(images))
        dev_end = int(0.9 * len(images))
        
        random.shuffle(images)
        
        train = images[:train_end]
        dev = images[train_end:dev_end]
        test = images[dev_end:]
        
        # Copy images into corresponding directories.
        for image_dir, images_in_dir in zip([train_dir, dev_dir, test_dir], [train, dev, test]):
            for image in images_in_dir:
                dst = os.path.join(os.path.join(image_dir, name), os.path.basename(image))
                shutil.copyfile(image, dst)

In [25]:
transf = transforms.Compose([transforms.Resize((224, 224)), transforms.RandomRotation(20), 
                             transforms.RandomPerspective(), transforms.ToTensor()])

# Creating datasets for every folder.
train_folder = ImageFolder(root=train_dir, transform=transf)
dev_folder = ImageFolder(root=dev_dir, transform=transf)
test_folder = ImageFolder(root=test_dir, transform=transf)

# Creating loaders for every dataset.
train_loader = DataLoader(train_folder, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_folder, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_folder, batch_size=BATCH_SIZE, shuffle=True)

# Showing example of coded labels.
images, labels = iter(train_loader).next()
print(labels)

tensor([ 3, 30, 18,  9])


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py:440: UserWarning: torch.gels is deprecated in favour of torch.lstsq and will be removed in the next release. Please use torch.lstsq instead.
  res = torch.gels(B, A)[0]


### Trying different models.

In [26]:
# Simple convolution neural network (given by pytorch tutorial)

class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(16 * 56 * 56, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 33)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [27]:
# EfficientNet from zero to hero

efficientnet = efficientnet_pytorch.EfficientNet.from_name('efficientnet-b0')

In [34]:
# Pretrained EfficientNet as feature extractor + couple of Fully Connected Layer

class TransferedEfficientNet(nn.Module):
    def __init__(self):
        super(TransferedEfficientNet, self).__init__()
        self.feature_extractor = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b0')
        self.fc1 = nn.Linear(1280 * 7 * 7, 600)
        self.fc2 = nn.Linear(600, 33)
    
    def forward(self, x):
        x = self.feature_extractor.extract_features(x)
        x = x.view(-1, 1280 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [35]:
model = TransferedEfficientNet()  # Choose your fighter.
model.to(device)  # For GPU

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# print(model)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/checkpoints/efficientnet-b0-355c32eb.pth
100.0%


Loaded pretrained weights for efficientnet-b0


In [36]:
for epoch in range(20):  

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py:440: UserWarning: torch.gels is deprecated in favour of torch.lstsq and will be removed in the next release. Please use torch.lstsq instead.
  res = torch.gels(B, A)[0]


[1,   200] loss: 0.195304
[1,   400] loss: 0.085889
[1,   600] loss: 0.090332
[1,   800] loss: 0.108014
[1,  1000] loss: 0.137762
[2,   200] loss: 0.060981
[2,   400] loss: 0.080803
[2,   600] loss: 0.080575
[2,   800] loss: 0.041115
[2,  1000] loss: 0.045977
[3,   200] loss: 0.064417
[3,   400] loss: 0.118629
[3,   600] loss: 0.048417
[3,   800] loss: 0.053681
[3,  1000] loss: 0.023506
[4,   200] loss: 0.068049
[4,   400] loss: 0.068625
[4,   600] loss: 0.053555
[4,   800] loss: 0.053583
[4,  1000] loss: 0.037105
[5,   200] loss: 0.055722
[5,   400] loss: 0.049288
[5,   600] loss: 0.045770
[5,   800] loss: 0.058304
[5,  1000] loss: 0.090076
[6,   200] loss: 0.030096
[6,   400] loss: 0.042557
[6,   600] loss: 0.042707
[6,   800] loss: 0.027201
[6,  1000] loss: 0.021277
[7,   200] loss: 0.065125
[7,   400] loss: 0.034381
[7,   600] loss: 0.039413
[7,   800] loss: 0.072828
[7,  1000] loss: 0.045965
[8,   200] loss: 0.075963
[8,   400] loss: 0.027129
[8,   600] loss: 0.028814
[8,   800] l

### Saving trained model

In [37]:
MODEL_PATH = './trained_models'
os.makedirs(MODEL_PATH, exist_ok=True)
str_time = time.strftime("%Y-%m-%d_%H:%M_model.pth", time.gmtime())
torch.save(model.state_dict(), os.path.join(MODEL_PATH, str_time))

---

### Loading saved model

In [39]:
model = TransferedEfficientNet()
model.to(device)  # For GPU
model.load_state_dict(torch.load(os.path.join(MODEL_PATH, str_time)))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [40]:
# Testing accuracy

correct = 0
total = 0
with torch.no_grad():
    for data in dev_loader:
        images, labels = data[0].to(device), data[1].to(device)  # For GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the dev images: %d %%' % (100 * correct / total))

Accuracy on the dev images: 96 %


In [41]:
# Testing accuracy for each class

class_correct = [0.0 for i in range(33)]
class_total = [0.0 for i in range(33)]
with torch.no_grad():
    for data in dev_loader:
        images, labels = data[0].to(device), data[1].to(device)  # For GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(BATCH_SIZE):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(33):
    print('Accuracy of %43s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of                    1 гривня (зразка 2006 р) : 84 %
Accuracy of     1 гривня (зразка 2006 р, зворотнiй бiк) : 100 %
Accuracy of                    2 гривнi (зразка 2004 р) : 100 %
Accuracy of     2 гривнi (зразка 2004 р, зворотнiй бiк) : 100 %
Accuracy of                   5 гривень (зразка 2004 р) : 100 %
Accuracy of    5 гривень (зразка 2004 р, зворотнiй бiк) : 93 %
Accuracy of                  10 гривень (зразка 2004 р) : 100 %
Accuracy of   10 гривень (зразка 2004 р, зворотнiй бiк) : 100 %
Accuracy of                  20 гривень (зразка 2003 р) : 93 %
Accuracy of   20 гривень (зразка 2003 р, зворотнiй бiк) : 94 %
Accuracy of                  50 гривень (зразка 2004 р) : 100 %
Accuracy of   50 гривень (зразка 2004 р, зворотнiй бiк) : 100 %
Accuracy of                 100 гривень (зразка 2005 р) : 100 %
Accuracy of  100 гривень (зразка 2005 р, зворотнiй бiк) : 81 %
Accuracy of                 200 гривень (зразка 2007 р) : 100 %
Accuracy of  200 гривень (зразка 2007 р, звор